In [9]:
def get_llm():
    load_dotenv()
    llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key = os.environ["GOOGLE_API_KEY"],temperature = 0.1)
    return llm

In [11]:
def get_prompt():
    prompt_templet = """
    Given the following context and a question, generate an answer based on the context only.
    In the answer try to provide as much text as possible from "response" section in the source document.
    If the answer is not found in the context, kindly state that "Sorry  , need more training ". Don't try to make up an answer.

    CONTEXT: {context}

    QUESTION: {question}

    """

    prompt = PromptTemplate(
    template = prompt_templet,
    input_variables = ["context","question"]
    )
    
    return prompt

In [12]:
def vector_database():
    
    loader = CSVLoader(file_path=data_location,source_column="prompt",encoding='cp1252')
    data = loader.load()
    embeddings = HuggingFaceInstructEmbeddings()
    vectordb = FAISS.from_documents(documents = data, embedding = embeddings)
    vectordb.save_local(file_location)
    
    return embeddings

In [21]:
def get_embedding():
    embeddings = HuggingFaceInstructEmbeddings()
    return embeddings

In [16]:
def get_chain():
    
    llm = get_llm()
    embeddings = get_embedding()
    vectordb = FAISS.load_local(file,embeddings,allow_dangerous_deserialization=True)
    retriever = vectordb.as_retriever()
    prompt = get_prompt()
    
    chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff",
        retriever = retriever,
        input_key = "query",
        return_source_documents = True,
        chain_type_kwargs = {"prompt":prompt}
    
    )
    
    return chain

In [ ]:

import os
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
load_dotenv()

def add_data_to_vector_database(new_data_location,file_location):

    new_data_loader = CSVLoader(file_path=new_data_location, source_column="prompt", encoding='cp1252')
    new_data = new_data_loader.load()

    if os.path.exists(file_location):
        embeddings = HuggingFaceInstructEmbeddings()
        vectordb = FAISS.load_local(file_location, embeddings, allow_dangerous_deserialization=True)
        
        vectordb.add_documents(new_data)
        vectordb.save_local(file_location)
    else:
        print("Vector database not found. Please create a vector database first.")

file_location="vectorDB"
new_data_location = "new_data/c.csv"
add_data_to_vector_database(new_data_location,file_location)




In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
load_dotenv()

def add_data_to_vector_database(new_data_location, file_location):
    try:
        new_data_loader = CSVLoader(file_path=new_data_location, source_column="prompt", encoding='cp1252')
        new_data = new_data_loader.load()
        if os.path.exists(file_location):
            embeddings = HuggingFaceInstructEmbeddings()
            vectordb = FAISS.load_local(file_location, embeddings, allow_dangerous_deserialization=True)
            vectordb.add_documents(new_data)
            vectordb.save_local(file_location)
            print("Data successfully added to the vector database.")
        else:
            print("Vector database not found. Please create a vector database first.")
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")

file_location = "vectorDB"
new_data_location = "new_data/c.csv"
add_data_to_vector_database(new_data_location, file_location)

In [ ]:
import os
import time
import schedule
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
import logging
load_dotenv()

logging.basicConfig(filename='vector_update.log', level=logging.INFO, 
                    format='%(asctime)s - %(message)s')

def add_data_to_vector_database(new_data_location, file_location):
    try:
        new_data_loader = CSVLoader(file_path=new_data_location, source_column="prompt", encoding='cp1252')
        new_data = new_data_loader.load()

        if os.path.exists(file_location):
            embeddings = HuggingFaceInstructEmbeddings() 
            vectordb = FAISS.load_local(file_location, embeddings, allow_dangerous_deserialization=True)
            
            vectordb.add_documents(new_data)
            vectordb.save_local(file_location)
            logging.info(f"Data successfully added from {new_data_location} to the vector database at {file_location}.")
        else:
            logging.error("Vector database not found. Please create a vector database first.")
    
    except Exception as e:
        logging.error(f"An error occurred while updating the vector database: {str(e)}")

def periodic_update():
    file_location = "vectorDB" 
    new_data_location = "new_data/c.csv" 
    logging.info("Starting the periodic vector database update.")
    add_data_to_vector_database(new_data_location, file_location)

schedule.every(24).hours.do(periodic_update)

def run_scheduler():
    logging.info("Scheduler started.")
    while True:
        schedule.run_pending()
        time.sleep(1)

if __name__ == "__main__":
    run_scheduler()
